In [ ]:
!wget http://images.cocodataset.org/zips/val2014.zip
!wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Questions_Val_mscoco.zip
!wget https://s3.amazonaws.com/cvmlp/vqa/mscoco/vqa/v2_Annotations_Val_mscoco.zip

--2024-02-20 13:12:47--  http://images.cocodataset.org/zips/val2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.221.33, 3.5.11.147, 52.217.230.249, ...
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.221.33|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6645013297 (6.2G) [application/zip]
Saving to: ‘val2014.zip’

val2014.zip         100%[===================>]   6.19G  34.5MB/s    in 1m 52s  

2024-02-20 13:14:39 (56.5 MB/s) - ‘val2014.zip’ saved [6645013297/6645013297]



In [ ]:
!unzip -q val2014.zip

In [ ]:
!unzip -q /content/v2_Annotations_Val_mscoco.zip
!unzip -q /content/v2_Questions_Val_mscoco.zip

In [ ]:
import os
from tqdm import tqdm
import json

len(os.listdir("val2014"))

40504

In [ ]:


with open("/content/v2_mscoco_val2014_annotations.json") as f:
  annotation_data = json.load(f)

with open("/content/v2_OpenEnded_mscoco_val2014_questions.json") as f:
  question_data = json.load(f)

In [ ]:
question_id_dict = {}
for data in question_data["questions"]:
  question_id_dict[data["question_id"]] = data

In [ ]:
for data in annotation_data["annotations"]:
  data["question"] = question_id_dict[data["question_id"]]["question"]

In [ ]:
annotation_data["annotations"][0]

{'question_type': 'none of the above',
 'multiple_choice_answer': 'down',
 'answers': [{'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 1},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 2},
  {'answer': 'at table', 'answer_confidence': 'yes', 'answer_id': 3},
  {'answer': 'skateboard', 'answer_confidence': 'yes', 'answer_id': 4},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 5},
  {'answer': 'table', 'answer_confidence': 'yes', 'answer_id': 6},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 7},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 8},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 9},
  {'answer': 'down', 'answer_confidence': 'yes', 'answer_id': 10}],
 'image_id': 262148,
 'answer_type': 'other',
 'question_id': 262148000,
 'question': 'Where is he looking?'}

In [ ]:
directional_words = ["left", "right", "up", "down", "above", "below", "over", "under", "top", "bottom",
 "port", "starboard", "upward", "downward", "overhead", "beneath", "atop", "summit"]

In [ ]:
def forbidden_word_as_answer(answer_list, forbidden_words):
  for data in answer_list:
    for word in forbidden_words:
      if word in data["answer"]:
        return True
  return False

In [ ]:
from nltk.corpus import wordnet as wn
import nltk
nltk.download('wordnet')

def has_antonyms_wordnet(words):
  for word in words:
    for synset in wn.synsets(word):
      for lemma in synset.lemmas():
        if lemma.antonyms():
          if lemma.antonyms()[0].name() in words:
            return True
  return False

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
MIN_ANSWERS = 4
annotations_without_directional_words = []

for data in tqdm(annotation_data["annotations"]):
  if forbidden_word_as_answer(data["answers"], directional_words):
    continue

  new_data_format = {
      "question": data["question"],
      "question_id": data["question_id"],
      "image_id": data["image_id"],
      "answers": list(set([x["answer"] for x in data["answers"] if x["answer_confidence"]=="yes"]))
  }

  if len(new_data_format["answers"]) < MIN_ANSWERS:
    continue
  if has_antonyms_wordnet(new_data_format["answers"]):
    continue
  annotations_without_directional_words.append(new_data_format)

len(annotation_data["annotations"]), len(annotations_without_directional_words)

100%|██████████| 214354/214354 [00:09<00:00, 21913.92it/s]


(214354, 31958)

In [ ]:
import random
random.seed(53)

SAMPLES = 200
dataset = random.sample(annotations_without_directional_words, SAMPLES)

In [ ]:
!mkdir dataset4

mkdir: cannot create directory ‘dataset4’: File exists


In [ ]:
os.listdir("val2014")[0]

'COCO_val2014_000000512982.jpg'

In [ ]:
import shutil

for data in tqdm(dataset):
  image_path = "val2014/COCO_val2014_"+str(data["image_id"]).zfill(12)+".jpg"
  shutil.copy(image_path, "dataset4/"+str(data["image_id"])+".jpg")

100%|██████████| 200/200 [00:00<00:00, 2075.66it/s]


In [ ]:
with open("dataset4_eval.json","w") as f:
  json.dump(dataset, f, indent=4)

In [ ]:
!zip -rq dataset4.zip dataset4

In [ ]:
!curl --upload-file dataset4.zip https://transfer.sh/dataset4.zip

https://transfer.sh/RwUCilpbgq/dataset4.zip